# Module 1.1 - Introduction to oneDNN - Getting Started

## Learning Objectives
In this module the developer will:
* Learn different oneDNN releases inside the oneAPI toolkit
* Learn how to compile a oneDNN sample with different releases via batch jobs on the Intel oneAPI DevCloud
* Learn how to program oneDNN with a simple sample


***
# Getting Started Sample Exercise


## introduce oneDNN releases inside oneAPI toolkits
oneDNN has four different releases inside the oneAPI toolkits. Each release is in a different folder under the oneDNN installation path, and each release supports a different compiler or threading library.


Set the installation path of your oneAPI toolkit

In [1]:
%env ONEAPI_INSTALL=/home/intel/intel/inteloneapi/

env: ONEAPI_INSTALL=/home/intel/intel/inteloneapi/


In [2]:
!printf '%s\n'     $ONEAPI_INSTALL/oneDNN/latest/cpu_*

/home/intel/intel/inteloneapi//oneDNN/latest/cpu_dpcpp_gpu_dpcpp
/home/intel/intel/inteloneapi//oneDNN/latest/cpu_gomp
/home/intel/intel/inteloneapi//oneDNN/latest/cpu_iomp
/home/intel/intel/inteloneapi//oneDNN/latest/cpu_tbb


As you can see, there are four different folders under the oneDNN installation path, and each of those releases supports different features. This tutorial will guide you how to compile and run against different oneDNN releases.

First, create a lab folder for this exercise.

In [3]:
!mkdir lab

##  Editing the getting_started.cpp code
The Jupyter cell below with the gray background can be edited in-place and saved.

The first line of the cell contains the command **%%writefile 'lab/getting_started.cpp'** This tells the input cell to save the contents of the cell into the file name 'getting_started.cpp'  As you edit the cell and run it, it will save your changes into that file.

In [4]:
%%writefile lab/getting_started.cpp

/// This C++ API example demonstrates basics of DNNL programming
/// model.


#include <cmath>
#include <numeric>
#include <sstream>
#include <vector>

#include "dnnl_debug.h"
#include "example_utils.hpp"

using namespace dnnl;


void getting_started_tutorial(engine::kind engine_kind) {

    engine eng(engine_kind, 0);

    stream engine_stream(eng);

    const int N = 1, H = 13, W = 13, C = 3;

    // Compute physical strides for each dimension
    const int stride_N = H * W * C;
    const int stride_H = W * C;
    const int stride_W = C;
    const int stride_C = 1;

    // An auxiliary function that maps logical index to the physical offset
    auto offset = [=](int n, int h, int w, int c) {
        return n * stride_N + h * stride_H + w * stride_W + c * stride_C;
    };

    // The image size
    const int image_size = N * H * W * C;

    // Allocate a buffer for the image
    std::vector<float> image(image_size);

    // Initialize the image with some values
    for (int n = 0; n < N; ++n)
        for (int h = 0; h < H; ++h)
            for (int w = 0; w < W; ++w)
                for (int c = 0; c < C; ++c) {
                    int off = offset(
                            n, h, w, c); // Get the physical offset of a pixel
                    image[off] = -std::cos(off / 10.f);
                }

    auto src_md = memory::desc(
            {N, C, H, W}, // logical dims, the order is defined by a primitive
            memory::data_type::f32, // tensor's data type
            memory::format_tag::nhwc // memory format, NHWC in this case
    );

    auto alt_src_md = memory::desc(
            {N, C, H, W}, // logical dims, the order is defined by a primitive
            memory::data_type::f32, // tensor's data type
            {stride_N, stride_C, stride_H, stride_W} // the strides
    );

    // Sanity check: the memory descriptors should be the same
    if (src_md != alt_src_md)
        throw std::string("memory descriptor initialization mismatch");

    auto src_mem = memory(src_md, eng);
    write_to_dnnl_memory(image.data(), src_mem);

    auto dst_mem = memory(src_md, eng);

    auto relu_d = eltwise_forward::desc(
            prop_kind::forward_inference, algorithm::eltwise_relu,
            src_md, // the memory descriptor for an operation to work on
            0.f, // alpha parameter means negative slope in case of ReLU
            0.f // beta parameter is ignored in case of ReLU
    );

    // ReLU primitive descriptor, which corresponds to a particular
    // implementation in the library
    auto relu_pd
            = eltwise_forward::primitive_desc(relu_d, // an operation descriptor
                    eng // an engine the primitive will be created for
            );


    auto relu = eltwise_forward(relu_pd); // !!! this can take quite some time

    relu.execute(engine_stream, // The execution stream
            {
                    // A map with all inputs and outputs
                    {DNNL_ARG_SRC, src_mem}, // Source tag and memory obj
                    {DNNL_ARG_DST, dst_mem}, // Destination tag and memory obj
            });

    // Wait the stream to complete the execution
    engine_stream.wait();

    std::vector<float> relu_image(image_size);
    read_from_dnnl_memory(relu_image.data(), dst_mem);
    
    
    // Check the results
    for (int n = 0; n < N; ++n)
        for (int h = 0; h < H; ++h)
            for (int w = 0; w < W; ++w)
                for (int c = 0; c < C; ++c) {
                    int off = offset(
                            n, h, w, c); // get the physical offset of a pixel
                    float expected = image[off] < 0
                            ? 0.f
                            : image[off]; // expected value
                    if (relu_image[off] != expected) {
                        std::cout << "At index(" << n << ", " << c << ", " << h
                                  << ", " << w << ") expect " << expected
                                  << " but got " << relu_image[off]
                                  << std::endl;
                        throw std::logic_error("Accuracy check failed.");
                    }
                }
    // [Check the results]

}


int main(int argc, char **argv) {
    try {
        engine::kind engine_kind = parse_engine_kind(argc, argv);
        getting_started_tutorial(engine_kind);
        std::cout << "Example passes" << std::endl;
    } catch (dnnl::error &e) {
        std::cerr << "DNNL error: " << e.what() << std::endl
                  << "Error status: " << dnnl_status2str(e.status) << std::endl;
        return 1;
    } catch (std::string &e) {
        std::cerr << "Error in the example: " << e << std::endl;
        return 2;
    }

    return 0;
}
// [Main]


Writing lab/getting_started.cpp


Then, copy the required header files and CMake file into lab folder.

In [5]:
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/example_utils.hpp lab/
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/example_utils.h lab/
!cp $ONEAPI_INSTALL/oneDNN/latest/cpu_dpcpp_gpu_dpcpp/examples/CMakeLists.txt lab/

## DevCloud Build and Execution
This training is delivered and run on the Intel oneAPI DevCloud. To enable a large number of developers using the DevCloud simultaneously to enjoy the fullness of tools and access to a variety of hardware without delay, the DevCloud uses the Portable Batch System (PBS).

As such, users must employ PBS utilities such as **qsub**, **pbsnodes**, **qstat**, and others to request and use compute resources. For more information, refer to [Using Intel® DevCloud with oneAPI Product](https://software.intel.com/en-us/articles/using-intel-devcloud-with-oneapi-products).

For training purposes we have written script utilities to ease developers in using the PBS system.



### remove "source setvars.sh" from bash_profile for custom configuration of oneDNN
In order to switch between different oneDNN releases, the user must use a custom configuration when running "source setvars.sh". However, bash_profile does "source setvars.sh" without any configuration by default. In order to switch between different releases successfully, we must remove the "source setvars.sh" in ~/.bash_profile by running the following command.

In [ ]:
%%writefile ~/.bash_profile
# .bash_profile

# Get the aliases and functions
if [ -f ~/.bashrc ]; then
   . ~/.bashrc
fi

# User specific environment and startup programs
export PATH=$PATH:$HOME/.local/bin:$HOME/bin

# Enable Intel tools
export INTEL_LICENSE_FILE=/usr/local/licenseserver/psxe.lic
export PATH=/glob/intel-python/python3/bin/:/glob/intel-python/python2/bin/:${PATH}
source /glob/development-tools/parallel-studio/bin/compilervars.sh intel64
export PATH=$PATH:/bin
#if [ -d /opt/intel/inteloneapi ]; then source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1; fi

# Make sure that most programs (in particular, pip) leave temp files locally
if [ ! -d ${HOME}/tmp ]; then
  mkdir ${HOME}/tmp
fi
export TMPDIR=${HOME}/tmp
export ONEAPI_INSTALL=/opt/intel/inteloneapi/

# Build and Run with DPC++ release 
one of the oneDNN releases supports DPC++, and it can run on different architectures by using DPC++.
The following section guides you how to build with DPC++ and run on different architectures.

#### Script - build.sh
The script **build.sh** encapsulates the compiler **dpcpp** command and flags that will generate the exectuable.
In order to use DPC++ compiler and related SYCL runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ release : 

   -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=dpcpp -DDNNL_CPU_RUNTIME=SYCL -DDNNL_GPU_RUNTIME=SYCL

In [6]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir dpcpp
cd dpcpp
cmake .. -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=dpcpp -DDNNL_CPU_RUNTIME=SYCL -DDNNL_GPU_RUNTIME=SYCL
make getting-started-cpp



Writing build.sh


Once you achieve an all-clear from your compilation, you execute your program on the DevCloud.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
By default, the built program uses CPU as the execution engine, but the user can switch to GPU by giving an input argument "gpu".
The user can refer run.sh below to run on GPU.
To run on CPU, simply remove the input argument "gpu" .

In [7]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --force > /dev/null 2>&1
echo "########## Executing the run"
./dpcpp/out/getting-started-cpp cpu
echo "########## Done with the run"


Writing run.sh



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.


In [8]:
! rm -rf dpcpp;chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi

-- The C compiler identification is Clang 10.0.0
-- The CXX compiler identification is Clang 10.0.0
-- Check for working C compiler: /home/intel/intel/inteloneapi/compiler/latest/linux/bin/clang
-- Check for working C compiler: /home/intel/intel/inteloneapi/compiler/latest/linux/bin/clang -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /home/intel/intel/inteloneapi/compiler/latest/linux/bin/dpcpp
-- Check for working CXX compiler: /home/intel/intel/inteloneapi/compiler/latest/linux/bin/dpcpp -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CMAKE_BUILD_TYPE is unset, defaulting to Release
-- DNNLROOT: /home/intel/intel/inteloneapi/oneDNN/2021.1-beta04/cpu_dpcpp_gpu_dpcpp
-- Could NOT find OpenMP_C (missing: OpenMP_C_FLAGS OpenMP_C_LIB_N



### Build and Run with GNU Compiler with OpenMP release 
one of the oneDNN releases supports GNU compilers, but it can run only on CPU.
The following section guides you how to build with G++ and run on CPU.

#### Script - build.sh
The script **build.sh** encapsulates the compiler command and flags that will generate the exectuable.
The user must switch to the G++ oneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".
In order to use the G++ compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ release : 

  -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE

In [15]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_gomp --force> /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_gomp
cd cpu_gomp
cmake .. -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Overwriting build.sh


Once you achieve an all-clear from your compilation, you execute your program on the DevCloud.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the G++ oneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".

In [13]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_gomp --force> /dev/null 2>&1
echo "########## Executing the run"
./cpu_gomp/out/getting-started-cpp
echo "########## Done with the run"


Overwriting run.sh



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in non-DevCloud environments (locally).
To enable users to run their scripts both on the DevCloud and in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails, it is assumed that build/run will be local.

In [14]:
! rm -rf cpu_gomp;chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi

-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/gcc
-- Check for working C compiler: /usr/bin/gcc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/g++
-- Check for working CXX compiler: /usr/bin/g++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CMAKE_BUILD_TYPE is unset, defaulting to Release
-- DNNLROOT: /home/intel/intel/inteloneapi/oneDNN/2021.1-beta04/cpu_gomp
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Configuring done
-- Generating done
-- Build files have been written to: /home/intel/WORK/oneAPI/eco/DLDevKit-code-samples/one


### Build and Run with Intel Compiler with OpenMP release 
one of the oneDNN releases supports Intel compilers, but it can run only on CPU.
The following section guides you how to build with ICC and run on CPU.

#### Script - build.sh
The script **build.sh** encapsulates the compiler command and flags that will generate the exectuable.
The user must switch to the ICC oneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_iomp" when running "source setvars.sh".
In order to use ICC compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ release : 

  -DCMAKE_C_COMPILER=icc -DCMAKE_CXX_COMPILER=icpc -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_iomp --force> /dev/null 2>&1
#source ~/intel/inteloneapi/setvars.sh  --dnnl-configuration=cpu_iomp > /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_iomp
cd cpu_iomp
cmake .. -DCMAKE_C_COMPILER=icc -DCMAKE_CXX_COMPILER=icpc -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the ICC oneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_iomp" when running "source setvars.sh".

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_iomp > /dev/null 2>&1
echo "########## Executing the run"
./cpu_iomp/out/getting-started-cpp
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in non-DevCloud environments (locally).
To enable users to run their scripts both on the DevCloud and in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails it is assumed that build/run will be local.

In [ ]:
! chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi



### Build and Run with GNU Compiler with TBB release 
one of the oneDNN releases supports Intel Threading building block as its threading library, but it can run only on CPU.
The following section guides you how to build with TBB and run on CPU.

#### Script - build.sh
The script **build.sh** encapsulates the compiler **dpcpp** command and flags that will generate the exectuable.
The user must switch to the G++ oneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_gomp" when running "source setvars.sh".
In order to use G++ compiler and related OMP runtime, some definitions must be passed as cmake arguments.
Here are related cmake arguments for DPC++ release : 

  -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=OMP -DDNNL_GPU_RUNTIME=NONE

In [ ]:
%%writefile build.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_tbb > /dev/null 2>&1
export EXAMPLE_ROOT=./lab/
mkdir cpu_tbb
cd cpu_tbb
cmake .. -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DDNNL_CPU_RUNTIME=TBB -DDNNL_GPU_RUNTIME=NONE
make getting-started-cpp



Once you achieve an all-clear from your compilation, you execute your program on the DevCloud.

#### Script - run.sh
the script **run.sh** encapsulates the program for submission to the job queue for execution.
The user must switch to the TBBoneDNN release by inputting a custom configuration "--dnnl-configuration=cpu_tbb" when running "source setvars.sh".

In [ ]:
%%writefile run.sh
#!/bin/bash
source $ONEAPI_INSTALL/setvars.sh --dnnl-configuration=cpu_tbb --force> /dev/null 2>&1
echo "########## Executing the run"
./cpu_tbb/out/getting-started-cpp
echo "########## Done with the run"



#### Submitting **build.sh** and **run.sh** to the job queue
Now we can submit the **build.sh** and **run.sh** to the job queue.

##### NOTE - it is possible to execute any of the build and run commands in non-DevCloud environments (locally).
To enable users to run their scripts both on the DevCloud and in local environments, this and subsequent training checks for the existence of the job submission command **qsub**.  If the check fails, it is assumed that build/run will be local.

In [ ]:
! chmod 755 q; chmod 755 build.sh; chmod 755 run.sh;if [ -x "$(command -v qsub)" ]; then ./q build.sh; ./q run.sh; else ./build.sh; ./run.sh; fi

***
# Summary
In this lab the developer learned the following:
* What are the different oneDNN releases inside the oneAPI toolkits
* How to compile a oneDNN sample with different releases via batch jobs on the Intel oneAPI DevCloud
* How to program oneDNN with a simple sample


***
# Transition Back To Slides